Import all necessary libraries to use BERT

In [2]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

Matplotlib is building the font cache; this may take a moment.


We now need to set up the dataset so that it can be used with a TensorFlow model (i.e. BERT)

In [25]:
with open("/Users/rorybeals/Desktop/CS159/nlp-final-project/data/liar.data/train.tsv", "r") as infile, open("/Users/rorybeals/Desktop/CS159/nlp-final-project/data/liar.data/traincleaned.tsv", "w") as outfile:
    for line in infile:
        # Remove problematic quotes and extra line breaks
        clean_line = line.replace('"', '').replace('\n', '').replace('\r', '')
        outfile.write(clean_line + "\n")


In [29]:
def parse_tsv_line(line):
    # Define the column types
    column_defaults = [
    '',  # JSON file name
    '',  # Boolean flag as string
    '',  # Statement
    '',  # Topic
    '',  # Speaker
    '',  # Speaker's title
    '',  # State
    '',  # Party
    0,   # Truth count 1
    0,   # Truth count 2
    0,   # Truth count 3
    0,   # Truth count 4
    0,   # Truth count 5
    ''   # Source
]
    # Decode the line into individual columns
    columns = tf.io.decode_csv(line, record_defaults=column_defaults, field_delim='\t', use_quote_delim=True)
    # Separate features and label
    # Separate features and label (assuming no label for now)
    string_features = columns[2:8] + columns[-1:]  # Select all string columns
    numeric_features = columns[8:-1]  # Select integer columns (truth counts)

    # Combine string and numeric features into a single dictionary
    features = {
        "string_features": string_features,
        "numeric_features": tf.stack(numeric_features)
    }

    # If there's a label column, define it here; otherwise, return features only
    label = columns[1]  # Replace with actual label column index if applicable
    return features, label

def create_tsv_dataset(file_path, batch_size=32):
    # Load the file
    dataset = tf.data.TextLineDataset(file_path)
    # Skip the header if the file has one
    # dataset = dataset.skip(1) <- curret file doesn't have a header but if it did we could skip the top row
    # Parse each line
    dataset = dataset.map(parse_tsv_line)
    # Shuffle, batch, and prefetch for performance
    dataset = dataset.shuffle(buffer_size=1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset



In [32]:
# Path to your .tsv file
file_path = "/Users/rorybeals/Desktop/CS159/nlp-final-project/data/liar.data/traincleaned.tsv"

# Create the dataset
batch_size = 32

dataset = create_tsv_dataset(file_path, batch_size)

# Iterate through the dataset
for features, labels in dataset.take(1):
    print("Features:", features)
    print("Labels:", labels)


Features: {'string_features': <tf.Tensor: shape=(32, 7), dtype=string, numpy=
array([[b"I haven't missed a vote since 1993.", b'history',
        b'chuck-grassley', b'U.S. senator', b'Iowa', b'republican',
        b'a conference call'],
       [b'Says in the last 20 years, the French have not stood by the United States on foreign policy.',
        b'foreign-policy,history,military', b'rick-santorum', b'',
        b'Pennsylvania', b'republican', b'a campaign event'],
       [b'Wisconsin is one of the highest taxed states in the nation.',
        b'taxes', b'scott-walker', b'Milwaukee County Executive',
        b'Wisconsin', b'republican', b'a statement on campaign web site'],
       [b'They were going to build the wall a while ago, not so long ago, like in 06. And they couldnt get environmental impact statements.',
        b'immigration', b'donald-trump', b'President-Elect', b'New York',
        b'republican', b'an interview on Fox News'],
       [b'Florida teachers are already paid $5,

2024-11-25 14:35:43.596807: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
